In [ ]:
import pickle
import numpy as np
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
import torch.nn.functional as F
import json
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
data_path = "../../../Data/jigsaw/training_processed.pickle"
all_data = pickle.load(open(data_path, "rb"))

In [ ]:
print(len(all_data["all_data"]), len(all_data["positives"]), len(all_data["negatives"]))

In [ ]:
positive_cutoff = len(all_data["positives"])//10
negative_cutoff = len(all_data["negatives"])//10

testing_positives = all_data["positives"][-positive_cutoff:]
testing_negatives = all_data["negatives"][-negative_cutoff:]

testing_data = [(index, 1) for index in testing_positives]
testing_data.extend([(index, 0) for index in testing_negatives])

np.random.shuffle(testing_data)
print(testing_data[:100])

training_positives = all_data["positives"][:-positive_cutoff]
training_negatives = all_data["negatives"][:-negative_cutoff]

_data = all_data["all_data"]

In [ ]:
max_comment_length = 230 #99th percentile of comment lengths

In [ ]:
def genBatch(bs=8, testing = False, cuda=True):
    
    _positive_data = training_positives
    _negative_data = training_negatives
    
    _batch = None
    if (not testing):
        #during training always present a balanced training set
        positive_samples = bs//2
        negative_samples = bs - positive_samples
        _p = np.random.randint(0, len(_positive_data), (positive_samples,))
        _n = np.random.randint(0, len(_negative_data), (negative_samples,))

        _batch = [(tokenizer.convert_tokens_to_ids(_data[index]["comment_text"]), 1) 
                  for index in _p]
        _batch.extend([(tokenizer.convert_tokens_to_ids(_data[index]["comment_text"]), 0) 
                       for index in _n])
    else:
        _batch = []
        _indices = np.random.randint(0, len(testing_data), (bs,))
        for list_index in _indices:
            _index = testing_data[list_index][0]
            _class = testing_data[list_index][1]
            _batch.append((tokenizer.convert_tokens_to_ids(_data[_index]["comment_text"]), _class))
        
    np.random.shuffle(_batch)
    _docs = [dp[0] for dp in _batch]
    _y = [dp[1] for dp in _batch]
    
    docs = []
    for _doc in _docs:
        _doc.insert(0, 101)
        while (len(_doc) < max_comment_length):
            _doc.append(0)
        docs.append(_doc[:max_comment_length])
    
    docs = np.asarray(docs)
    segments = np.zeros(docs.shape)
    y = np.asarray(_y)
    if (cuda):
        docs = torch.LongTensor(docs).cuda()
        segments = torch.LongTensor(segments).cuda()
        y = torch.FloatTensor(y).cuda()
    else:
        docs = torch.LongTensor(docs)
        segments = torch.LongTensor(segments)
        y = torch.FloatTensor(y)
    mask = docs > 0
    
    return docs, segments, mask, y

d, se, m, y = genBatch(bs=12, testing=False, cuda=True)
print(d.size(), se.size(), m.size(), y.size())

In [ ]:
bert = BertModel.from_pretrained("bert-base-uncased")
bert.cuda()

In [ ]:
class AttentionHead(torch.nn.Module):
    def __init__(self, dim=64, bert_model = "bert-base-uncased"):
        super(AttentionHead, self).__init__()
        self.bert_dim = 768
        if ("large" in bert_model):
            self.bert_dim = 1024
        self.w = torch.nn.Linear(self.bert_dim, dim)
        self.v = torch.nn.Linear(dim,1)
        self.o = torch.nn.Linear(self.bert_dim, dim)
    
    def forward(self, _d):
        _att = torch.tanh(self.w(_d))
        _att = self.v(_att)
        _att = F.softmax(_att, dim=1)
        _o = _d * _att
        _o = torch.sum(_o, dim=1)
        return self.o(_o)

In [ ]:
class multiHeadedClassifier(torch.nn.Module):
    def __init__(self, attention_heads = 8, 
                 attention_head_dim=64,
                 bert_model = "bert-base-uncased",
                 output_dims = 1
                ):
        super(multiHeadedClassifier, self).__init__()
        self.attentions = torch.nn.ModuleList([])
        self.bert_dim = 768
        if ("large" in bert_model):
            self.bert_dim = 1024
        self.bert = BertModel.from_pretrained("bert-base-uncased").cuda()
        for i in range(attention_heads):
            self.attentions.append(AttentionHead(dim=attention_head_dim, bert_model=bert_model))
        self.output = torch.nn.Linear(self.bert_dim + attention_heads*attention_head_dim, 1)
    
    def forward(self, d, se, m):
        _d, _ = self.bert(d, se, m, output_all_encoded_layers=False)
        att_outs = []
        for i in range(len(self.attentions)):
            head_out = self.attentions[i](_d)
            att_outs.append(head_out)
        att_outs = torch.cat(att_outs, dim=-1).unsqueeze(1)
        return self.output(torch.cat([_d[:,0,:].unsqueeze(1), att_outs], dim=-1)).squeeze(-1)

In [ ]:
network = multiHeadedClassifier()
network.cuda()

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optim = torch.optim.SGD(network.parameters(), lr=1e-2)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score as roc
def validate(network, batches=10, bs=12):
    print("\tValidating...")
    y_preds = []
    y_acts = []
    for i in range(batches):
        with torch.no_grad():
            d, se, m, y = genBatch(bs=12, testing=True, cuda=True)
            y_pred = network.forward(d, se, m)
            if (len(y_preds) == 0):
                y_preds = y_pred
                y_acts = y
            else:
                y_preds = torch.cat([y_preds, y_pred], dim=0)
                y_acts = torch.cat([y_acts, y])

    loss = loss_fn(y_preds, y_acts.unsqueeze(-1))
    score = roc(y_acts.cpu().numpy(), y_preds.cpu().numpy())
    print("\t", np.round(loss.data.item(), 5), np.round(score,2))
    return loss.data.item(), score

In [ ]:
validation_losses = []
epoch_losses = []
validation_rocs = []

def _save(network, cause):
    print("\tSaving model for cause:", cause)
    torch.save(network.state_dict(), "./UnbiasedToxicity_" + cause + ".h5")
    _trgcycle = {
        "training_losses":epoch_losses,
        "validation_losses":validation_losses,
        "roc_auc":validation_rocs
    }
    with open("./TrainingCycle_UbiasedToxicity_" + cause + ".json", "w") as f:
        f.write(json.dumps(_trgcycle))
        f.close()
    
def saveModel(network):
    loss, roc_auc = validate(network)
    validation_losses.append(loss)
    validation_rocs.append(roc_auc)
    
    if (np.min(epoch_losses) == epoch_losses[-1]):
        _save(network,"BestTrainingLoss")
    
    if (np.min(validation_losses) == validation_losses[-1]):
        _save(network, "BestValidationLoss")
    
    if (np.max(validation_rocs) == validation_rocs[-1]):
        _save(network, "BestRoCAUC")

In [ ]:
def train(network, optimizer=None, loss=None, epochs=2, batches_per_epoch=10, bs=12):
    for k in range(epochs):
        batch_losses = []
        for j in range(batches_per_epoch):
            optimizer.zero_grad()
            d, se, m, y = genBatch(bs=bs, testing=False, cuda=True)
            y_pred = network.forward(d, se, m)
            loss = loss_fn(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            y = y.detach().cpu().numpy()
            y_ = torch.sigmoid(y_pred).detach().cpu().numpy()
            batch_losses.append(loss.data.item())
            _str = "Epoch: " + str(k + 1) + "; Batch: (" + str(j+1) + "/" + str(batches_per_epoch) + ")"
            _str = _str + "\tLoss (AUC last batch): " + str(np.round(np.mean(batch_losses), 5)) + \
                    "(" + str(np.round(roc(y,y_), 2)) + ")"
            print(_str, end="\r")
        print("\n")
        epoch_losses.append(np.mean(batch_losses))
        saveModel(network)

In [ ]:
train(network, optimizer=optim, loss=loss_fn, epochs=25, bs=8, batches_per_epoch=2500)